<p style="text-align: center;"><span style="color: #ff0000;"><strong><span style="font-size: x-large;">
    ANEXO XXX: ALGORITMOS PKE</span></strong></span></p>

<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Realizado por:</span></strong></span></p>
<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Gabriel Vacaro Goytia</span></strong></span></p>
<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Ignacio Warleta Murcia</span></strong></span></p>

Este notebook contiene cuatro implementaciones de ataques con el objetivo de simular los pasos que un atacante seguiría para romper la seguridad de un esquema de cifrado Kyber-KEM.

Organizamos el anexo según el siguiente índice:

# Índice

1. [Introducción](#1.-Introducción)
2. [Configuración previa](#2.-Configuracion-previa)
3. [Algoritmo de generación de claves](#3.-Algoritmo-de-generacion-de-claves)
4. [Algoritmo de cifrado](#4.-Algoritmo-de-cifrado)
5. [Algoritmo de descifrado](#5.-Algoritmo-de-descifrado)
6. [Ejemplo de uso](#6.-Ejemplo-de-uso)

---
# 1. Introducción






Este notebook cuenta 4 formas de atacar el sistema Kyber-KEM:

- Ataque ingenuo <br>
- Ataque basado en aproximación con mínimos cuadrados <br>
- Ataque de reducción de redes
- Ataque basado en aprendizaje automático

El propósito de esta implementación es simula un intento de ataque en el que el atacante conoce los datos $(A, b)$ y pretende averiguar $s$. Primero, se genera un par de claves públicas y privada mediante la creación de una matriz $A$ y un vector secreto $s$, con un pequeño error $e$. La clave pública consta de $A$ y $b = A * s + e$ $mod$ $q$, mientras que la clave privada es $s$. 


---
# 2. Configuracion previa

A continuación, se muestra la configuración previa a ejecutar y los parámetros a definir.

In [1]:
#MODULOS A IMPORTAR
import numpy as np
import secrets
from scipy.linalg import lstsq
from itertools import product
from scipy.spatial import distance
from sklearn.linear_model import LogisticRegression

In [97]:
# Parámetros básicos
q = 749  # Un número primo pequeño típico en Kyber es 3329
k = 3  # Tamaño del vector/matriz (varía según los estándares Kyber-512, 768, 1024)
mu_1 = 2.0  # Parámetro para la distribución de error más controlado (menor desviación estándar)

In [56]:
# Función para generar una semilla aleatoria
def generate_seed(n=16):
    return secrets.token_bytes(n)

# Función para generar la matriz A de forma determinista
def generate_matrix_A(seed, k, q):
    np.random.seed(int.from_bytes(seed, "big") % (2**32))
    return np.random.randint(0, q, size=(k, k))

# Función para muestrear errores con distribución normal
def sample_error(mu, k, q, seed=None):
    if seed is not None:
        np.random.seed(int.from_bytes(seed, "big") % (2**32))
    return np.round(np.random.normal(0, mu, size=(k, 1))).astype(int) % q

# Generación de claves públicas y privadas
def key_generation():
    seed_A = generate_seed()  # Generamos una semilla para la matriz A
    A = generate_matrix_A(seed_A, k, q)  # Matriz A generada con la semilla
    s = sample_error(mu_1, k, q)  # Vector secreto s
    e = sample_error(mu_1, k, q)  # Vector de error e
    b = (A @ s + e) % q  # Cálculo de b = A * s + e mod q
    return (b, seed_A), s


---
## 3. Ataque ingenuo

Este código implementa un ataque ingenuo para intentar recuperar la clave secreta $s$ en el esquema Kyber resolviendo directamente el sistema de ecuaciones lineales $A * s = b $ $mod$ $q$. Utiliza la pseudoinversa de la matriz  $A$ (`np.linalg.pinv(A)`) para calcular una aproximación de $s$, redondea los valores y aplica la operación módulo $q$ para mantenerlos en el rango adecuado. Si la matriz $A$ no es invertible, se captura la excepción y se informa del error. Este método es poco efectivo en la práctica, ya que no tiene en cuenta la presencia de ruido (vector de error $e$), lo que dificulta la recuperación precisa de $s$.


In [57]:
# Ataque ingenuo: Resolver A * s = b mod q
def naive_attack(A, b):
    """
    Implementa un ataque ingenuo para intentar recuperar la clave secreta s 
    resolviendo directamente la ecuación A * s = b mod q.

    Parámetros:
    - A: Matriz pública generada en el esquema de cifrado.
    - b: Vector resultado de la operación A * s + e mod q.

    Retorna:
    - Un vector estimado de s en el dominio ℤ_q si el ataque tiene éxito.
    - None en caso de error (por ejemplo, si la matriz A no es invertible).
    """
    print("\nIntento de ataque ingenuo: Resolver el sistema A * s = b mod q")
    try:
        # Se calcula la pseudoinversa de A para estimar s
        s_recovered = np.linalg.pinv(A) @ b  
        
        # Se redondea y se reduce módulo q para obtener valores en el rango correcto
        s_recovered = np.round(s_recovered) % q

        # Se muestra el resultado obtenido
        print("Clave recuperada por el ataque ingenuo:")
        print(s_recovered)

        return s_recovered.astype(int)
    except np.linalg.LinAlgError:
        # En caso de que la matriz A no sea invertible, se informa el error
        print("Error: No se pudo invertir la matriz A.")
        return None


---
## 4. Ataque basado en aproximación con mínimos cuadrados

Este código implementa un ataque basado en la aproximación de mínimos cuadrados para intentar recuperar la clave secreta $s$. En este método, se usa la función `lstsq` de `scipy.linalg`, que resuelve el sistema $A* s ≈ b$ minimizando el error cuadrático. Luego, los valores obtenidos se redondean y se aplican módulo $q$ para ajustarlos al dominio de los coeficientes. Este enfoque es más robusto que el ataque ingenuo, pero aún puede ser ineficaz debido a la presencia del vector de error $e$, que introduce ruido en el sistema.

In [58]:
# Ataque basado en aproximación con mínimos cuadrados
def lattice_attack(A, b):
    """
    Implementa un ataque basado en la aproximación de mínimos cuadrados para 
    intentar recuperar la clave secreta s resolviendo A * s ≈ b mod q.

    Parámetros:
    - A: Matriz pública utilizada en el esquema de cifrado.
    - b: Vector resultado de la operación A * s + e mod q.

    Retorna:
    - Un vector estimado de s en el dominio ℤ_q.
    """
    print("\nIntento de ataque usando aproximación de mínimos cuadrados")

    # Se utiliza el método de mínimos cuadrados para encontrar la mejor aproximación de s
    s_recovered, _, _, _ = lstsq(A, b)

    # Se redondea y se reduce módulo q para obtener valores dentro del rango correcto
    s_recovered = np.round(s_recovered) % q

    # Se muestra el resultado obtenido
    print("Clave recuperada por el ataque de mínimos cuadrados:")
    print(s_recovered)

    return s_recovered.astype(int)


---
## 5. Ataque de reducción de reticulos

Este código implementa un ataque basado en la reducción de reticulos utilizando una aproximación similar a LLL. Se amplía la matriz $A$ añadiendo una identidad escalada por $q$ para construir una base extendida. Luego, se utiliza el método de mínimos cuadrados (`lstsq`) para aproximar la solución del sistema extendido, donde se concatena $b$ con un vector de ceros. Finalmente, se redondean los primeros $k$ elementos y se aplica módulo $q$ para obtener una estimación de la clave secreta $s$. Este método explota propiedades de la teoría de redes para mejorar la aproximación en comparación con ataques más simples.


In [59]:
# Ataque de reducción de redes
def lattice_reduction_attack(A, b, q):
    """
    Implementa un ataque basado en la reducción de redes, utilizando una aproximación LLL.
    Este método intenta recuperar la clave secreta s explotando la estructura de la red 
    generada por la ecuación A * s ≈ b mod q.

    Parámetros:
    - A: Matriz pública utilizada en el esquema de cifrado.
    - b: Vector resultado de la operación A * s + e mod q.
    - q: Módulo primo utilizado en el esquema.

    Retorna:
    - Un vector estimado de s en el dominio ℤ_q.
    """
    print("\nIntento de ataque por reducción de redes (aproximación LLL)")

    # Se expande la matriz A con una identidad escalada por q para formar la base de la red
    A_expanded = np.hstack((A, np.identity(k) * q))

    # Se utiliza la técnica de mínimos cuadrados para aproximar la solución en la red
    s_recovered, _, _, _ = lstsq(A_expanded, np.hstack((b, np.zeros((k, 1)))))

    # Se toman solo los primeros k elementos y se reducen módulo q
    s_recovered = np.round(s_recovered[:k]) % q

    # Se muestra el resultado obtenido
    print("Clave recuperada por reducción de redes:")
    print(s_recovered)

    return s_recovered.astype(int)


---
## 6. Ataque basado en aprendizaje automático

Este código implementa un ataque basado en aprendizaje automático utilizando regresión logística. Se emplea la matriz $A$ como conjunto de características de entrada ($X$) y el vector $b$ como etiquetas de salida ($y$). Se entrena un modelo de regresión logística con el «solver 'lbfgs'», configurado para un máximo de 1000 iteraciones. Una vez entrenado, el modelo predice los valores de la clave secreta $s$, que luego se ajustan al módulo $q$. Este enfoque intenta aprender una relación aproximada entre $A$ y $b$ para inferir la clave secreta basándose en patrones detectados en los datos.


In [60]:
# Ataque basado en aprendizaje automático
def machine_learning_attack(A, b):
    """
    Implementa un ataque basado en aprendizaje automático utilizando regresión logística.
    El objetivo es entrenar un modelo para predecir la clave secreta s a partir de la matriz A 
    y el vector b generado en el esquema LWE.

    Parámetros:
    - A: Matriz pública utilizada como conjunto de características.
    - b: Vector resultado de la operación A * s + e mod q.

    Retorna:
    - Un vector estimado de s en el dominio ℤ_q.
    """
    print("\nIntento de ataque por aprendizaje automático")

    X = A  # La matriz A actúa como el conjunto de características
    y = b.ravel()  # El vector b se usa como etiquetas de salida

    # Se entrena un modelo de regresión logística para intentar inferir la clave secreta
    model = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='auto')
    model.fit(X, y)

    # Se predice el vector s y se reduce módulo q
    s_recovered = model.predict(X).reshape(k, 1) % q

    # Se muestra la clave estimada
    print("Clave recuperada por aprendizaje automático:")
    print(s_recovered)

    return s_recovered.astype(int)


---
## 7. Simulación de ataques

Este fragmento de código ejecuta una simulación de ataques contra el esquema de cifrado basado en Kyber. Primero, genera un par de claves: la clave pública $(b, \text{seed}_A)$ y la clave secreta $s$. Luego, utiliza la matriz $A$ derivada de la semilla para intentar recuperar la clave secreta a través de diferentes métodos de ataque:

1. **Ataque ingenuo**: Usa la pseudoinversa de $A$ para estimar $s$.
2. **Ataque de mínimos cuadrados**: Aplica la aproximación de mínimos cuadrados para resolver $A \cdot s = b$.
3. **Ataque por reducción de redes**: Expande la matriz $A$ y aplica reducción de redes para mejorar la estimación de $s$.
4. **Ataque basado en aprendizaje automático**: Utiliza regresión logística para aprender una relación entre $A$ y $b$ y predecir $s$.

Finalmente, el código muestra la clave secreta real y compara los resultados de cada ataque con la clave original, indicando cuáles lograron recuperarla con éxito.


In [98]:
def print_header(title):
    print("\n" + "=" * 40)
    print(f"{title.center(40)}")
    print("=" * 40 + "\n")

def print_result(label, success):
    status = "✅ Éxito" if success else "❌ Fallo"
    print(f"{label.ljust(45)} {status}")

# Simulación del ataque
print_header("SIMULACIÓN DEL ATAQUE")
(pk, sk) = key_generation()
b, seed_A = pk
A = generate_matrix_A(seed_A, k, q)

# Ataques
print_header("EJECUTANDO ATAQUES")
s_attempt_naive = naive_attack(A, b)
s_attempt_lattice = lattice_attack(A, b)
s_attempt_lattice_reduction = lattice_reduction_attack(A, b, q)
s_attempt_ml = machine_learning_attack(A, b)

# Mostrar la clave secreta real
print_header("CLAVE SECRETA REAL")
print(sk)

# Comparación con la clave recuperada
print_header("COMPARACIÓN DE RESULTADOS")
print_result("Ataque ingenuo", np.array_equal(sk, s_attempt_naive))
print_result("Ataque de mínimos cuadrados", np.array_equal(sk, s_attempt_lattice))
print_result("Ataque de reducción de redes", np.array_equal(sk, s_attempt_lattice_reduction))
print_result("Ataque de aprendizaje automático", np.array_equal(sk, s_attempt_ml))

print("\n" + "=" * 40)



         SIMULACIÓN DEL ATAQUE          


           EJECUTANDO ATAQUES           


Intento de ataque ingenuo: Resolver el sistema A * s = b mod q
Clave recuperada por el ataque ingenuo:
[[  4.]
 [748.]
 [  1.]]

Intento de ataque usando aproximación de mínimos cuadrados
Clave recuperada por el ataque de mínimos cuadrados:
[[  4.]
 [748.]
 [  1.]]

Intento de ataque por reducción de redes (aproximación LLL)
Clave recuperada por reducción de redes:
[[0. 0.]
 [0. 0.]
 [0. 0.]]

Intento de ataque por aprendizaje automático
Clave recuperada por aprendizaje automático:
[[205]
 [362]
 [145]]

           CLAVE SECRETA REAL           

[[  1]
 [748]
 [  0]]

       COMPARACIÓN DE RESULTADOS        

Ataque ingenuo                                ❌ Fallo
Ataque de mínimos cuadrados                   ❌ Fallo
Ataque de reducción de redes                  ❌ Fallo
Ataque de aprendizaje automático              ❌ Fallo



/opt/miniconda3/envs/env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


---
A continuacion, puede probar a cambiar los parámetros y ver sus efectos en los ataques.

Algunas posibles opciones de los parámetros siguientes para ver el éxito de los ataques pueden ser:
- Primos como: 43	83 103 131 157 191 223 263 283	
- Tamaño del vector: 2, 1
- Error: 1.0
  
Con estas opciones el atacante debería tener más probabilidades de éxito.

In [100]:
# Parámetros básicos
q = 109  # Un número primo pequeño típico en Kyber es 3329
k = 2  # Tamaño del vector/matriz (varía según los estándares Kyber-512, 768, 1024)
mu_1 = 1.0  # Parámetro para la distribución de error más controlado (menor desviación estándar)

In [103]:
def print_header(title):
    print("\n" + "=" * 40)
    print(f"{title.center(40)}")
    print("=" * 40 + "\n")

def print_result(label, success):
    status = "✅ Éxito" if success else "❌ Fallo"
    print(f"{label.ljust(45)} {status}")

# Simulación del ataque
print_header("SIMULACIÓN DEL ATAQUE")
(pk, sk) = key_generation()
b, seed_A = pk
A = generate_matrix_A(seed_A, k, q)

# Ataques
print_header("EJECUTANDO ATAQUES")
s_attempt_naive = naive_attack(A, b)
s_attempt_lattice = lattice_attack(A, b)
s_attempt_lattice_reduction = lattice_reduction_attack(A, b, q)
s_attempt_ml = machine_learning_attack(A, b)

# Mostrar la clave secreta real
print_header("CLAVE SECRETA REAL")
print(sk)

# Comparación con la clave recuperada
print_header("COMPARACIÓN DE RESULTADOS")
print_result("Ataque ingenuo", np.array_equal(sk, s_attempt_naive))
print_result("Ataque de mínimos cuadrados", np.array_equal(sk, s_attempt_lattice))
print_result("Ataque de reducción de redes", np.array_equal(sk, s_attempt_lattice_reduction))
print_result("Ataque de aprendizaje automático", np.array_equal(sk, s_attempt_ml))

print("\n" + "=" * 40)


         SIMULACIÓN DEL ATAQUE          


           EJECUTANDO ATAQUES           


Intento de ataque ingenuo: Resolver el sistema A * s = b mod q
Clave recuperada por el ataque ingenuo:
[[1.]
 [0.]]

Intento de ataque usando aproximación de mínimos cuadrados
Clave recuperada por el ataque de mínimos cuadrados:
[[1.]
 [0.]]

Intento de ataque por reducción de redes (aproximación LLL)
Clave recuperada por reducción de redes:
[[0. 0.]
 [0. 0.]]

Intento de ataque por aprendizaje automático
Clave recuperada por aprendizaje automático:
[[ 2]
 [26]]

           CLAVE SECRETA REAL           

[[1]
 [0]]

       COMPARACIÓN DE RESULTADOS        

Ataque ingenuo                                ✅ Éxito
Ataque de mínimos cuadrados                   ✅ Éxito
Ataque de reducción de redes                  ❌ Fallo
Ataque de aprendizaje automático              ❌ Fallo



/opt/miniconda3/envs/env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
